#📌 Extracción

In [15]:
# Extracción de datos desde la API (GitHub Raw) y conversión a DataFrame
# Se intenta primero con pandas.read_json (suficiente para JSON estructurado en lista / dict).
# Si se necesitara más control, se muestra también la alternativa con requests.

import pandas as pd
import json
from pathlib import Path

RAW_URL = "https://raw.githubusercontent.com/alura-cursos/challenge2-data-science-LATAM/main/TelecomX_Data.json"

# Directorio centralizado de datos
DATA_DIR = Path("data")
DATA_DIR.mkdir(parents=True, exist_ok=True)

try:
    df = pd.read_json(RAW_URL)
except ValueError:
    # Fallback más robusto si el JSON está anidado y read_json directo falla
    import requests
    resp = requests.get(RAW_URL, timeout=30)
    resp.raise_for_status()
    data = resp.json()
    # Intentamos detectar estructura
    if isinstance(data, list):
        df = pd.DataFrame(data)
    elif isinstance(data, dict):
        # Si es un dict con una clave principal que contiene la lista de registros
        # Buscamos la primera lista dentro
        list_candidate = None
        for v in data.values():
            if isinstance(v, list):
                list_candidate = v
                break
        if list_candidate is not None:
            df = pd.DataFrame(list_candidate)
        else:
            # Último recurso: normalización completa
            from pandas import json_normalize
            df = json_normalize(data)
    else:
        raise RuntimeError("Formato JSON no soportado para conversión a DataFrame")

# Información básica
print("Filas, Columnas:", df.shape)
print("Columnas:")
print(df.columns.tolist())
print("\nTipos:")
print(df.dtypes.head())

# Vista rápida
display(df.head())

# (Opcional) Guardar copia local para trabajo offline
OUTPUT_FILE = DATA_DIR / "TelecomX_Data_local.parquet"
try:
    df.to_parquet(OUTPUT_FILE, index=False)
    print(f"Archivo guardado localmente en {OUTPUT_FILE.resolve()}")
except Exception as e:
    print("No se pudo guardar en Parquet (puede faltar pyarrow o fastparquet).", e)


Filas, Columnas: (7267, 6)
Columnas:
['customerID', 'Churn', 'customer', 'phone', 'internet', 'account']

Tipos:
customerID    object
Churn         object
customer      object
phone         object
internet      object
dtype: object


,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


Archivo guardado localmente en C:\Users\USER\Documents\data-science-especialización\challenges\Challenge Telecom X análisis de evasión de clientes\data\TelecomX_Data_local.parquet


#🔧 Transformación

In [17]:
# Exploración rápida de estructura antes de expandir diccionarios
# Asegúrate de haber ejecutado la celda de Extracción para tener 'df'
if 'df' not in globals():
    raise RuntimeError("Primero ejecuta la celda de Extracción para crear el DataFrame 'df'.")

print("== Forma inicial ==")
print(df.shape)
print("\n== Tipos de datos (df.dtypes) ==")
print(df.dtypes)
print("\n== df.info() ==")
# Usar buffer para que aparezca ordenado en Jupyter
import io, sys
buf = io.StringIO()
# verbose=True para listado completo
df.info(buf=buf, verbose=True, show_counts=True)
info_str = buf.getvalue()
print(info_str)

# Resumen compacto de conteo de tipos
import pandas as pd
summary_types = df.dtypes.value_counts()
print("== Conteo de tipos ==")
print(summary_types)

# (Opcional) Mostrar primeras filas sólo para inspección
display(df.head())

== Forma inicial ==
(7267, 6)

== Tipos de datos (df.dtypes) ==
customerID    object
Churn         object
customer      object
phone         object
internet      object
account       object
dtype: object

== df.info() ==
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   customerID  7267 non-null   object
 1   Churn       7267 non-null   object
 2   customer    7267 non-null   object
 3   phone       7267 non-null   object
 4   internet    7267 non-null   object
 5   account     7267 non-null   object
dtypes: object(6)
memory usage: 340.8+ KB

== Conteo de tipos ==
object    6
Name: count, dtype: int64


,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


In [18]:
# === Exploración y selección preliminar de variables para evasión (churn) ===
import pandas as pd
import numpy as np
from pathlib import Path

# 1. Recuperar DataFrame transformado si no está en memoria
if 'df_model' not in globals():
    parquet_path = Path('data') / 'TelecomX_transformado.parquet'
    if parquet_path.exists():
        df_model = pd.read_parquet(parquet_path)
        print(f"df_model cargado desde {parquet_path}")
    else:
        raise RuntimeError("No se encontró df_model ni el archivo transformado. Ejecuta las celdas anteriores.")

print(f"Dimensión df_model: {df_model.shape}")

# 2. Identificar columna objetivo (target) posible: buscar 'Churn' (case-insensitive)
candidate_target_cols = [c for c in df_model.columns if 'churn' in c.lower()]
print("Columnas que parecen target (contienen 'Churn'):", candidate_target_cols)
if not candidate_target_cols:
    raise RuntimeError("No se detectó columna objetivo con 'Churn'. Verifica nombres.")
# Si hubiese más de una, elegir la última (o personalizar manualmente)
TARGET_COL = candidate_target_cols[-1]
print(f"Usando '{TARGET_COL}' como variable objetivo.")

# Asegurar binaria 0/1
if df_model[TARGET_COL].dtype == 'O':
    df_model[TARGET_COL] = df_model[TARGET_COL].str.strip().str.title().replace({'Yes':1,'No':0})

# 3. Construir "diccionario" derivado de prefijos (si la estructura original venía agrupada)
feature_groups = {}
for col in df_model.columns:
    if col == TARGET_COL: continue
    prefix = col.split('_')[0] if '_' in col else 'misc'
    feature_groups.setdefault(prefix, []).append(col)
print("Grupos de variables por prefijo (diccionario derivado):")
for k,v in feature_groups.items():
    print(f"  {k}: {len(v)} columnas")

# 4. Resumen de tipos, nulos, cardinalidad
summary_rows = []
for col in df_model.columns:
    ser = df_model[col]
    summary_rows.append({
        'col': col,
        'dtype': str(ser.dtype),
        'n_null': ser.isna().sum(),
        '%null': ser.isna().mean()*100,
        'n_unique': ser.nunique(),
        'sample_vals': ser.dropna().unique()[:5]
    })
summary_df = pd.DataFrame(summary_rows).sort_values('%null', ascending=False)
print("\nResumen de columnas (top 10 por mayor % de nulos):")
display(summary_df.head(10))

# 5. Separar tipo numérico vs categórico (excluyendo target)
Y = df_model[TARGET_COL]
feature_cols = [c for c in df_model.columns if c != TARGET_COL]
num_cols = [c for c in feature_cols if pd.api.types.is_numeric_dtype(df_model[c]) and df_model[c].nunique()>2]
cat_cols = [c for c in feature_cols if c not in num_cols]
print(f"Numéricas: {len(num_cols)} | Categóricas: {len(cat_cols)}")

# 6. Métricas de relevancia simples
relevance_records = []
# Numéricas: correlación punto-biserial (equivalente a Pearson con binaria)
for col in num_cols:
    ser = df_model[col]
    if ser.isna().all():
        continue
    corr = ser.corr(Y)
    relevance_records.append({'col': col, 'type': 'num', 'metric': abs(corr), 'detail': corr})

# Categóricas: diferencia máxima de tasa de churn entre categorías
for col in cat_cols:
    ser = df_model[col].fillna('NA')
    # Asegurar no cardinalidad extrema (evitar IDs)
    if ser.nunique() > 40:
        continue
    rates = Y.groupby(ser).mean()
    impact = rates.max() - rates.min()
    relevance_records.append({'col': col, 'type': 'cat', 'metric': impact, 'detail': rates.to_dict()})

relevance_df = pd.DataFrame(relevance_records).sort_values('metric', ascending=False)
print("\nTop 15 variables por métrica simple de relevancia:")
display(relevance_df.head(15))

# 7. (Opcional) Mutual Information para corroborar (si sklearn disponible)
try:
    from sklearn.feature_selection import mutual_info_classif
    from sklearn.preprocessing import OrdinalEncoder
    X_mi = df_model[feature_cols].copy()
    # Relleno simple
    for c in X_mi.columns:
        if pd.api.types.is_numeric_dtype(X_mi[c]):
            X_mi[c] = X_mi[c].fillna(X_mi[c].median())
        else:
            X_mi[c] = X_mi[c].fillna('NA')
    # Codificación ordinal para categóricas
    enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
    X_enc = X_mi.copy()
    if cat_cols:
        X_enc[cat_cols] = enc.fit_transform(X_enc[cat_cols])
    mi_scores = mutual_info_classif(X_enc, Y, discrete_features=[c in cat_cols for c in X_enc.columns], random_state=42)
    mi_df = pd.DataFrame({'col': X_enc.columns, 'mutual_info': mi_scores}).sort_values('mutual_info', ascending=False)
    print("\nMutual Information (top 15):")
    display(mi_df.head(15))
except Exception as e:
    print("Saltando Mutual Information (instala scikit-learn si lo necesitas):", e)

# 8. Selección preliminar de columnas relevantes (umbral simple)
important_cols = relevance_df.query('metric > 0.02')['col'].tolist()  # umbral heurístico
print(f"\nColumnas preliminarmente relevantes (metric > 0.02): {len(important_cols)}")
print(important_cols)

# Guardar artefactos de exploración
summary_df.to_csv('data/summary_columns.csv', index=False)
relevance_df.to_csv('data/relevance_simple.csv', index=False)
if 'mi_df' in locals():
    mi_df.to_csv('data/relevance_mutual_info.csv', index=False)
print("Archivos de exploración guardados en carpeta data.")

# Nota: Ajusta criterios según necesites (umbral, excluir colinealidad, etc.)

Dimensión df_model: (7267, 21)
Columnas que parecen target (contienen 'Churn'): ['Churn']
Usando 'Churn' como variable objetivo.
Grupos de variables por prefijo (diccionario derivado):
  misc: 1 columnas
  customer: 5 columnas
  phone: 2 columnas
  internet: 7 columnas
  account: 5 columnas

Resumen de columnas (top 10 por mayor % de nulos):


,col,dtype,n_null,%null,n_unique,sample_vals
1,Churn,float64,224,3.082427,2,"[0.0, 1.0]"
0,customerID,object,0,0.000000,7267,"[0002-ORFBO, 0003-MKNFE, 0004-TLHLJ, 0011-IGKF..."
2,customer_gender,object,0,0.000000,2,"[Female, Male]"
3,customer_SeniorCitizen,int64,0,0.000000,2,"[0, 1]"
4,customer_Partner,int64,0,0.000000,2,"[1, 0]"
5,customer_Dependents,int64,0,0.000000,2,"[1, 0]"
6,customer_tenure,int64,0,0.000000,73,"[9, 4, 13, 3, 71]"
7,phone_PhoneService,int64,0,0.000000,2,"[1, 0]"
8,phone_MultipleLines,object,0,0.000000,3,"[No, Yes, No phone service]"
9,internet_InternetService,object,0,0.000000,3,"[DSL, Fiber optic, No]"


Numéricas: 2 | Categóricas: 18

Top 15 variables por métrica simple de relevancia:


,col,type,metric,detail
15,account_Contract,cat,0.398778,"{'Month-to-month': 0.4270967741935484, 'One ye..."
0,customer_tenure,num,0.352229,-0.352229
8,internet_InternetService,cat,0.344878,"{'DSL': 0.1895910780669145, 'Fiber optic': 0.4..."
9,internet_OnlineSecurity,cat,0.343617,"{'No': 0.4176672384219554, 'No internet servic..."
12,internet_TechSupport,cat,0.342305,"{'No': 0.4163547365390153, 'No internet servic..."
10,internet_OnlineBackup,cat,0.325238,"{'No': 0.39928756476683935, 'No internet servi..."
11,internet_DeviceProtection,cat,0.317226,"{'No': 0.3912762520193861, 'No internet servic..."
17,account_PaymentMethod,cat,0.300423,{'Bank transfer (automatic)': 0.16709844559585...
14,internet_StreamingMovies,cat,0.262755,"{'No': 0.33680430879712747, 'No internet servi..."
13,internet_StreamingTV,cat,0.261182,"{'No': 0.33523131672597867, 'No internet servi..."


Saltando Mutual Information (instala scikit-learn si lo necesitas): No module named 'sklearn'

Columnas preliminarmente relevantes (metric > 0.02): 16
['account_Contract', 'customer_tenure', 'internet_InternetService', 'internet_OnlineSecurity', 'internet_TechSupport', 'internet_OnlineBackup', 'internet_DeviceProtection', 'account_PaymentMethod', 'internet_StreamingMovies', 'internet_StreamingTV', 'account_Charges.Monthly', 'customer_SeniorCitizen', 'account_PaperlessBilling', 'customer_Dependents', 'customer_Partner', 'phone_MultipleLines']
Archivos de exploración guardados en carpeta data.


In [20]:


if 'df_model' not in globals():
    path_parquet = Path('data') / 'TelecomX_transformado.parquet'
    if not path_parquet.exists():
        raise RuntimeError("Ejecuta antes las celdas de extracción y transformación.")
    df_model = pd.read_parquet(path_parquet)

print("Dimensión:", df_model.shape)

# 1. Duplicados
n_dup_rows = df_model.duplicated().sum()
print(f"Filas duplicadas (completas): {n_dup_rows}")

# 2. Resumen de nulos
null_summary = (df_model.isna().sum()
                .to_frame('n_null')
                .assign(pct=lambda d: d.n_null/len(df_model)*100)
                .sort_values('pct', ascending=False))
print("\nTop columnas con nulos:")
display(null_summary.head(15))

# Guardar resumen de nulos
null_summary.to_csv('data/quality_nulls.csv')

# 3. Detección de columnas object que parecen numéricas (posibles errores de formato)
candidate_numeric = []
for c in df_model.select_dtypes(include='object').columns:
    ser = df_model[c].dropna()
    if ser.empty: 
        continue
    # proporción de valores que se convierten a número
    conv = pd.to_numeric(ser, errors='coerce')
    ratio = conv.notna().mean()
    if ratio > 0.95:  # umbral
        candidate_numeric.append((c, ratio))
if candidate_numeric:
    print("\nColumnas object que son casi numéricas (revisar conversión):")
    for c, r in candidate_numeric:
        print(f"  {c} -> {r:.1%} convertibles")

# 4. Inconsistencias de categorías (espacios / mayúsculas-minúsculas)
cat_issues = []
for c in df_model.select_dtypes(include='object').columns:
    ser = df_model[c].dropna()
    if ser.empty:
        continue
    normalized = ser.str.strip().str.lower()
    if normalized.nunique() < ser.nunique():
        diff = ser.nunique() - normalized.nunique()
        cat_issues.append((c, diff, ser.nunique(), normalized.nunique()))
if cat_issues:
    print("\nColumnas con posibles inconsistencias de formato en categorías:")
    for c, diff, orig_u, norm_u in cat_issues:
        print(f"  {c}: {orig_u} -> {norm_u} (reducción {diff})")
    # Ejemplo de corrección (no se aplica automáticamente para que revises):
    # df_model['col'] = df_model['col'].str.strip().str.title()

# 5. Fechas: intentar detectar columnas convertibles a datetime
datetime_candidates = []
for c in df_model.columns:
    if df_model[c].dtype == 'object':
        sample = df_model[c].dropna()
        if sample.empty:
            continue
        parsed = pd.to_datetime(sample, errors='coerce', utc=False, format=None)
        ratio = parsed.notna().mean()
        if ratio > 0.85:
            datetime_candidates.append((c, ratio))
if datetime_candidates:
    print("\nColumnas que podrían ser datetime (revisar):")
    for c, r in datetime_candidates:
        print(f"  {c}: {r:.1%} parseable")
    # Ejemplo:
    # df_model['fecha_normalizada'] = pd.to_datetime(df_model['fecha'], errors='coerce').dt.normalize()

# 6. Cardinalidad alta (posibles IDs a excluir del modelado)
high_card = []
for c in df_model.columns:
    uniq = df_model[c].nunique(dropna=True)
    if  uniq > 0.9 * len(df_model) and uniq > 50:
        high_card.append(c)
if high_card:
    print("\nColumnas de cardinalidad muy alta (posibles identificadores):")
    print(high_card)

# 7. Resumen final a CSV
quality_report = {
    'n_rows': [len(df_model)],
    'n_cols': [df_model.shape[1]],
    'duplicate_rows': [n_dup_rows],
    'cols_with_nulls': [(null_summary.query('n_null>0').shape[0])],
    'possible_numeric_objects': [len(candidate_numeric)],
    'category_inconsistency_cols': [len(cat_issues)],
    'datetime_candidates': [len(datetime_candidates)],
    'high_cardinality_cols': [len(high_card)]
}
pd.DataFrame(quality_report).to_csv('data/quality_overview.csv', index=False)
print("\nArchivos guardados: data/quality_nulls.csv, data/quality_overview.csv")


Dimensión: (7267, 21)
Filas duplicadas (completas): 0

Top columnas con nulos:


,n_null,pct
Churn,224,3.082427
customerID,0,0.000000
customer_gender,0,0.000000
customer_SeniorCitizen,0,0.000000
customer_Partner,0,0.000000
customer_Dependents,0,0.000000
customer_tenure,0,0.000000
phone_PhoneService,0,0.000000
phone_MultipleLines,0,0.000000
internet_InternetService,0,0.000000



Columnas object que son casi numéricas (revisar conversión):
  account_Charges.Total -> 99.8% convertibles


C:\Users\USER\AppData\Local\Temp\ipykernel_15472\1046560576.py:64: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed = pd.to_datetime(sample, errors='coerce', utc=False, format=None)



Columnas de cardinalidad muy alta (posibles identificadores):
['customerID']

Archivos guardados: data/quality_nulls.csv, data/quality_overview.csv


C:\Users\USER\AppData\Local\Temp\ipykernel_15472\1046560576.py:64: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed = pd.to_datetime(sample, errors='coerce', utc=False, format=None)
C:\Users\USER\AppData\Local\Temp\ipykernel_15472\1046560576.py:64: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed = pd.to_datetime(sample, errors='coerce', utc=False, format=None)
C:\Users\USER\AppData\Local\Temp\ipykernel_15472\1046560576.py:64: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed = pd.to_datetime(sample, errors='coerce', utc=False, format=None)
C:\Users\USER\AppData\

In [19]:
# Detectar y expandir columnas que contienen diccionarios anidados
import pandas as pd
import numpy as np
from pathlib import Path

# Asegurar directorio de datos coherente con la fase de extracción
DATA_DIR = Path("data")
DATA_DIR.mkdir(parents=True, exist_ok=True)

# Asumimos que el DataFrame original se llama df (creado en la fase de extracción)
# Si vienes a esta celda sin haber ejecutado la anterior, vuelve y ejecútala primero.

# 1. Identificar columnas con valores tipo dict
nested_cols = [c for c in df.columns if df[c].apply(lambda x: isinstance(x, dict)).any()]
print(f"Columnas anidadas detectadas: {nested_cols}")

# 2. Función para expandir de forma segura cada columna dict
from pandas import json_normalize

def expand_dict_columns(dataframe, cols, prefix=True):
    out_df = dataframe.copy()
    for col in cols:
        # Normalizamos; ignoramos filas que no sean dict sustituyéndolas por {} para evitar errores
        safe_series = out_df[col].apply(lambda x: x if isinstance(x, dict) else {})
        expanded = json_normalize(safe_series)
        # Prefijos para evitar colisiones de nombres
        if prefix:
            expanded = expanded.add_prefix(f"{col}_")
        # Unir y eliminar la original
        out_df = pd.concat([out_df.drop(columns=[col]), expanded], axis=1)
    return out_df

expanded_df = expand_dict_columns(df, nested_cols, prefix=True)
print("Forma original:", df.shape, "-> Forma expandida:", expanded_df.shape)

# 3. Tipificación básica (ejemplos)
# Convertir TotalCharges si existe y está como texto
for candidate in ["account_TotalCharges", "TotalCharges", "account_totalCharges", "account_Totalcharges"]:
    if candidate in expanded_df.columns:
        expanded_df[candidate] = pd.to_numeric(expanded_df[candidate], errors='coerce')

# Mapear Yes/No habituales a 1/0 (sin tocar columnas que no sean objet/boolean)
yes_no_map = {"Yes": 1, "No": 0, "Yes ": 1, "No ": 0}
for col in expanded_df.select_dtypes(include=['object']).columns:
    uniques = set(expanded_df[col].dropna().unique())
    if uniques.issubset(set(yes_no_map.keys()) | {"", " "}):
        expanded_df[col] = expanded_df[col].map(yes_no_map)

# 4. Mostrar vista previa
display(expanded_df.head())

# 5. Guardar resultado intermedio en carpeta data
TRANSFORM_FILE = DATA_DIR / "TelecomX_transformado.parquet"
expanded_df.to_parquet(TRANSFORM_FILE, index=False)
print(f"Dataset transformado guardado en {TRANSFORM_FILE.resolve()}")

# Dejamos el DataFrame transformado en una variable principal para siguientes pasos
df_model = expanded_df.copy()
print("Variable df_model lista para análisis/modelado.")

Columnas anidadas detectadas: ['customer', 'phone', 'internet', 'account']
Forma original: (7267, 6) -> Forma expandida: (7267, 21)


,customerID,Churn,customer_gender,customer_SeniorCitizen,customer_Partner,customer_Dependents,customer_tenure,phone_PhoneService,phone_MultipleLines,internet_InternetService,...,internet_OnlineBackup,internet_DeviceProtection,internet_TechSupport,internet_StreamingTV,internet_StreamingMovies,account_Contract,account_PaperlessBilling,account_PaymentMethod,account_Charges.Monthly,account_Charges.Total
0,0002-ORFBO,0.0,Female,0,1,1,9,1,No,DSL,...,Yes,No,Yes,Yes,No,One year,1,Mailed check,65.6,593.3
1,0003-MKNFE,0.0,Male,0,0,0,9,1,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,0,Mailed check,59.9,542.4
2,0004-TLHLJ,1.0,Male,0,0,0,4,1,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,1,Electronic check,73.9,280.85
3,0011-IGKFF,1.0,Male,1,1,0,13,1,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,1,Electronic check,98.0,1237.85
4,0013-EXCHZ,1.0,Female,1,1,0,3,1,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,1,Mailed check,83.9,267.4


Dataset transformado guardado en C:\Users\USER\Documents\data-science-especialización\challenges\Challenge Telecom X análisis de evasión de clientes\data\TelecomX_transformado.parquet
Variable df_model lista para análisis/modelado.


#📊 Carga y análisis

#📄Informe final